In [1]:
import warnings
warnings.filterwarnings('ignore') # somente para ignorar os warnings
import pandas as pd # ferramenta para ciencia de dados
import requests
import time
from tqdm import tqdm_notebook as tqdm # barra de progresso
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2

In [2]:
def conecta_db():
    con = psycopg2.connect(host='ec2-3-213-228-206.compute-1.amazonaws.com', 
                         database='dbola8cfs4iili',
                         user='mljmaxtzayxmfi',
                         password = 'f06107039bb077725d49b11b67a04e5bc3a70a4761fbbc3b859111f6a267a174',
                         port = 5432)
    return con

In [33]:
def inserir_db(sql,params):
    con = conecta_db()
    cur = con.cursor()
    #try:
    cur.execute(sql,params)
    con.commit()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         con.rollback()
#         cur.close()
#         return 1
    cur.close()

In [5]:
df = pd.read_csv('exemplo_dados_tratados.csv')

In [8]:
df = df.drop(columns = ['Unnamed: 0'])

In [14]:
df

,CEP,Nome,Bairro,Estado,Área total,Situação,Nome empresarial,coordenadas
0,71535000,,SETOR DE HABITACOES INDIVIDUAIS NORTE,DF,3931.19,2,IGREJA BATISTA DO LAGO NORTE,-15.7942287 -47.8821658
1,71746004,FERNANDO MONTEIRO RIBEIRO,PARK WAY,DF,580.00,14,NaN,-15.7753629 -47.8620279
2,73130900,THIAGO DE ANDRADE ALBUQUERQUE e Outros,ALTO DA BOA VISTA (SOBRADINHO),DF,265.06,14,NaN,-15.6356505 -47.7577427
3,73350350,EDIFICARE ENGENHARIA E CONSTRUCOES EIRELI,SETOR RESIDENCIAL LESTE (PLANALTINA),DF,1287.00,14,EDIFICARE ENGENHARIA E CONSTRUCOES EIRELI,-15.6212945 -47.6460883
4,72146000,"EVOLUCAO ENGENHARIA, CONSTRUCAO E ADMINISTRACA...",TAGUATINGA NORTE (TAGUATINGA),DF,1583.25,2,EVOLUCAO ENGENHARIA CONSTRUCAO E ADMINISTRACAO...,-15.8253882 -48.1201194
...,...,...,...,...,...,...,...,...
340,71525205,RN BLUMENSCHEIN LTDA,SETOR DE HABITACOES INDIVIDUAIS NORTE,DF,447.75,2,RN BLUMENSCHEIN LTDA,-15.7942287 -47.8821658
341,72001239,JMD ENGENHARIA EIRELI,SETOR HABITACIONAL VICENTE PIRES - TRECHO 3,DF,1631.00,2,JMD ENGENHARIA EIRELI,-15.8032275 -48.0175522
342,72135903,M.I CONSTRUTORA E SERVICOS EIRELI,SETOR INDUSTRIAL (TAGUATINGA),DF,5000.02,2,M.I CONSTRUTORA E SERVICOS EIRELI,-15.8472427 -48.044491
343,73130900,ALOISIO ARARUNA DE ALMEIDA JUNIOR,ALTO DA BOA VISTA (SOBRADINHO),DF,228.66,2,NaN,-15.6356505 -47.7577427


In [11]:
df['Nome'] = df['Nome'].fillna('')

In [15]:
df['Nome empresarial'] = df['Nome empresarial'].fillna('')

In [17]:
df['Bairro'] = df['Bairro'].fillna('')

In [12]:
df['Nome'] = df['Nome'].apply(lambda x: x.replace("'",''))

In [16]:
df['Nome empresarial'] = df['Nome empresarial'].apply(lambda x: x.replace("'",''))

In [18]:
df['Bairro'] = df['Bairro'].apply(lambda x: x.replace("'",''))

In [36]:
df['coordenadas'] = df['coordenadas'].apply(lambda x: str(list(reversed(x.split(' ')))).replace('[','').replace(']','').replace(',',' ').replace("'",''))

In [37]:
df

,CEP,Nome,Bairro,Estado,Área total,Situação,Nome empresarial,coordenadas
0,71535000,,SETOR DE HABITACOES INDIVIDUAIS NORTE,DF,3931.19,2,IGREJA BATISTA DO LAGO NORTE,-47.8821658 -15.7942287
1,71746004,FERNANDO MONTEIRO RIBEIRO,PARK WAY,DF,580.00,14,,-47.8620279 -15.7753629
2,73130900,THIAGO DE ANDRADE ALBUQUERQUE e Outros,ALTO DA BOA VISTA (SOBRADINHO),DF,265.06,14,,-47.7577427 -15.6356505
3,73350350,EDIFICARE ENGENHARIA E CONSTRUCOES EIRELI,SETOR RESIDENCIAL LESTE (PLANALTINA),DF,1287.00,14,EDIFICARE ENGENHARIA E CONSTRUCOES EIRELI,-47.6460883 -15.6212945
4,72146000,"EVOLUCAO ENGENHARIA, CONSTRUCAO E ADMINISTRACA...",TAGUATINGA NORTE (TAGUATINGA),DF,1583.25,2,EVOLUCAO ENGENHARIA CONSTRUCAO E ADMINISTRACAO...,-48.1201194 -15.8253882
...,...,...,...,...,...,...,...,...
340,71525205,RN BLUMENSCHEIN LTDA,SETOR DE HABITACOES INDIVIDUAIS NORTE,DF,447.75,2,RN BLUMENSCHEIN LTDA,-47.8821658 -15.7942287
341,72001239,JMD ENGENHARIA EIRELI,SETOR HABITACIONAL VICENTE PIRES - TRECHO 3,DF,1631.00,2,JMD ENGENHARIA EIRELI,-48.0175522 -15.8032275
342,72135903,M.I CONSTRUTORA E SERVICOS EIRELI,SETOR INDUSTRIAL (TAGUATINGA),DF,5000.02,2,M.I CONSTRUTORA E SERVICOS EIRELI,-48.044491 -15.8472427
343,73130900,ALOISIO ARARUNA DE ALMEIDA JUNIOR,ALTO DA BOA VISTA (SOBRADINHO),DF,228.66,2,,-47.7577427 -15.6356505


In [38]:
pbar = tqdm(total = len(df))

for i in df.index:
    sql = """
    INSERT into public.prototipo_construcoes (cep,nome,bairro,uf,area_total,situacao,nome_empresa,geom_novo) 
    values(%s,%s,%s,%s,%s,%s,%s,ST_GeomFromText(%s, 4326));
    """
    params = [str(df['CEP'][i]), df['Nome'][i], df['Bairro'][i], df['Estado'][i], float(df['Área total'][i]), int(df['Situação'][i]), df['Nome empresarial'][i],'POINT({})'.format(df['coordenadas'][i])]
    inserir_db(sql,params)
    
    pbar.update(1)

  0%|          | 0/345 [00:00<?, ?it/s]